In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')



In [2]:
df = pd.read_csv('corona_tested_individuals_ver_0083.english.csv')
df.head()

test_date  cough  fever  sore_throat  shortness_of_breath  head_ache  \
0  2020-11-12      0      0            0                    0          0   
1  2020-11-12      0      1            0                    0          0   
2  2020-11-12      0      0            0                    0          0   
3  2020-11-12      0      0            0                    0          0   
4  2020-11-12      0      1            0                    0          0   

  corona_result age_60_and_above  gender         test_indication  
0      negative               No    male                   Other  
1      negative               No    male                   Other  
2      negative              Yes  female                   Other  
3      negative               No    male                   Other  
4      negative               No    male  Contact with confirmed

In [3]:
df.shape

(2742596, 10)

In [4]:
df['test_indication'].value_counts()

Other                     2547559
Contact with confirmed     170742
Abroad                      24295
Name: test_indication, dtype: int64

In [5]:
gender_mapping = {'female': 0, 'male': 1}
df['gender'] = df['gender'].map(gender_mapping)

In [6]:
corona_result_mapping = {'positive': 1, 'negative': 0}
df['corona_result'] = df['corona_result'].map(corona_result_mapping)

In [7]:
age_60_and_above_mapping = {'Yes': 1, 'No': 0}
df['age_60_and_above'] = df['age_60_and_above'].map(age_60_and_above_mapping)

In [8]:
df['Contact_with_confirmed'] = df['test_indication'].apply(lambda x: 1 if x == 'Contact with confirmed' else 0)

In [9]:
print('Minimum Date',df['test_date'].min())
print('Maximum  Date',df['test_date'].max())

Minimum Date 2020-03-11
Maximum  Date 2020-11-12


In [10]:
df = df[df['test_date'] > '2020-09-01']

In [11]:
df.head()

test_date  cough  fever  sore_throat  shortness_of_breath  head_ache  \
0  2020-11-12      0      0            0                    0          0   
1  2020-11-12      0      1            0                    0          0   
2  2020-11-12      0      0            0                    0          0   
3  2020-11-12      0      0            0                    0          0   
4  2020-11-12      0      1            0                    0          0   

   corona_result  age_60_and_above  gender         test_indication  \
0            0.0               0.0     1.0                   Other   
1            0.0               0.0     1.0                   Other   
2            0.0               1.0     0.0                   Other   
3            0.0               0.0     1.0                   Other   
4            0.0               0.0     1.0  Contact with confirmed   

   Contact_with_confirmed  
0                       0  
1                       0  
2                       0  
3                       0  
4                       1

In [12]:
from sklearn.model_selection import train_test_split
df.sort_values(by='test_date', inplace=True)
test_size = 0.3
train_df, test_df = train_test_split(df, test_size=test_size, shuffle=False)
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

Training set shape: (855513, 11)
Testing set shape: (366649, 11)


In [13]:
train_df = train_df.drop(columns=['test_date','test_indication'])
test_df = test_df.drop(columns=['test_date','test_indication'])

In [14]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [15]:
from joblib import Parallel, delayed

X_train = train_df.drop(columns=['corona_result'])
y_train = train_df['corona_result']

X_test = test_df.drop(columns=['corona_result'])
y_test = test_df['corona_result']

# Random Forest
rf_param_grid = {
    'n_estimators': [25, 50],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Logistic Regression
lr_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 1, 10]
}

# Initialize classifiers
rf_classifier = RandomForestClassifier()
lr_classifier = LogisticRegression()
nb_classifier = GaussianNB()

# Define a function to perform grid search and return the fitted model
def perform_grid_search(classifier, param_grid, X_train, y_train):
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search

# Parallelize grid search for Random Forest and Logistic Regression
results = Parallel(n_jobs=-1)(delayed(perform_grid_search)(
    classifier, param_grid, X_train, y_train) for classifier, param_grid in [(rf_classifier, rf_param_grid), (lr_classifier, lr_param_grid)])

rf_grid_search, lr_grid_search = results  # Unpack the results

# Fit Naive Bayes directly
nb_classifier.fit(X_train, y_train)

# Predictions
rf_predictions = rf_grid_search.predict(X_test)
lr_predictions = lr_grid_search.predict(X_test)
nb_predictions = nb_classifier.predict(X_test)

# Print accuracy and classification report for each model
print("Random Forest Model:")
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print("Classification Report:\n", classification_report(y_test, rf_predictions))

print("\nLogistic Regression Model:")
print("Accuracy:", accuracy_score(y_test, lr_predictions))
print("Classification Report:\n", classification_report(y_test, lr_predictions))

print("\nNaive Bayes Model:")
print("Accuracy:", accuracy_score(y_test, nb_predictions))
print("Classification Report:\n", classification_report(y_test, nb_predictions))

Random Forest Model:
Accuracy: 0.9593876006117814
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.97      0.98    311872
         1.0       0.46      0.69      0.55     11773

    accuracy                           0.96    323645
   macro avg       0.72      0.83      0.77    323645
weighted avg       0.97      0.96      0.96    323645


Logistic Regression Model:
Accuracy: 0.9657371502726753
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.98      0.98    311872
         1.0       0.53      0.48      0.50     11773

    accuracy                           0.97    323645
   macro avg       0.76      0.73      0.74    323645
weighted avg       0.96      0.97      0.96    323645


Naive Bayes Model:
Accuracy: 0.9520338642648581
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    311872
         